#  **Welcome my friends**

**Import libraries**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import seaborn as sns
import random
import cv2 as cv
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout, Activation
from keras import optimizers
from keras_preprocessing.image import ImageDataGenerator
from keras.models import load_model

**Read csv file**

In [ ]:
df = pd.read_csv('../input/english-handwritten-characters-dataset/english.csv')
df.head()

**What are the classes?**

In [ ]:
classes = df['label'].unique()
quantity = len(df['label'].unique())
print(f'\n The %d Classes are: \n {classes}' %quantity)    

**Create validation and testing dataset**

In [ ]:
#Create a variable that contains the general file path
DATAPATH = '../input/english-handwritten-characters-dataset'
#Read another time the csv file
dataset = pd.read_csv(DATAPATH + '/english.csv')
#Pick randomly 600 values
rand = random.sample(range(len(dataset)), 300)
#Create validation set of data
validation_set = pd.DataFrame(dataset.iloc[rand,:].values, columns = ['image','label'])
#Drop Validation data from dataset
dataset.drop(rand, inplace = True)
#Pick randomly 15 values
rand = random.sample(range(len(validation_set)), 15)
#Create test set of data
test_set = pd.DataFrame(validation_set.iloc[rand,:].values, columns = ['image','label'])
#Drop Test data from validation set
validation_set.drop(rand, inplace = True)
#Show Validation dataset
validation_set

Data Augmentation

In [ ]:
train_data_generator = ImageDataGenerator(
            rescale=1/255,
            shear_range=0.2,
            zoom_range=0.2,
            width_shift_range=0.2,
            height_shift_range=0.2,)
data_generator = ImageDataGenerator(rescale = 1/255)
training_data_frame = train_data_generator.flow_from_dataframe( 
                                            dataframe = dataset, 
                                            directory = DATAPATH,
                                            x_col = 'image',
                                            y_col = 'label',
                                            target_size = (64,64),
                                            classe_mode = 'categorical')
validation_data_frame = data_generator.flow_from_dataframe( 
                                            dataframe = validation_set, 
                                            directory = DATAPATH + '',
                                            x_col = 'image',
                                            y_col = 'label',
                                            target_size = (64,64),
                                            classe_mode = 'categorical')
test_data_frame = data_generator.flow_from_dataframe( 
                                            dataframe = test_set, 
                                            directory = DATAPATH,
                                            x_col = 'image',
                                            y_col = 'label',
                                            target_size = (64,64),
                                            classe_mode = 'categorical')

In [ ]:
model = keras.Sequential([
        layers.BatchNormalization(),
        # Add first Convolutional Layer
        layers.Conv2D(filters=32, kernel_size=3, activation='relu',
                      padding='same', input_shape=(64,64,3)),
        # Add a second Convolutional Layer
        layers.Conv2D(filters=32, kernel_size=3, activation='relu'),
        # Add a Max pooling layer
        layers.MaxPool2D(pool_size=(2, 2)),
        # Add a Dropout layer
        layers.Dropout(0.25),
    
        layers.BatchNormalization(),
        # Add third Convolutional Layer
        layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding = 'same'),
        # Add Fourth Convolutional Layer
        layers.Conv2D(filters=32, kernel_size=3, activation='relu'),
        # Add a Max pooling layer
        layers.MaxPool2D(pool_size=(2, 2)),
        # Add a Dropout Layer
        layers.Dropout(0.25),

        layers.BatchNormalization(),
        # Add Fifth Convolutional Layer
        layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding = 'same'),
        # Add a sixth Convolutional Layer
        layers.Conv2D(filters=32, kernel_size=3, activation='relu'),
        # Add a Max Pooling Layer
        layers.MaxPool2D(pool_size=(2, 2)),
        # Add a Dropout Layer
        layers.Dropout(0.25),

        layers.BatchNormalization(),
        # Add a Flatten Layer
        layers.Flatten(),
        # Add a Dense layer Layer
        layers.Dense(512, activation = 'relu'),
        # Add a Dropout Layer
        layers.Dropout(0.5),
        # Add the Output Dense Layer
        layers.Dense(62, activation='softmax')
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(epsilon=0.01),
    loss= 'categorical_crossentropy',
    metrics = ['accuracy'])

**Train the module**

In [ ]:
history = model.fit(training_data_frame, validation_data = validation_data_frame, 
                    epochs = 50
                )

In [ ]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(50)

plt.figure(figsize=(15, 15))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, accuracy, label='Training Accuracy')
plt.plot(epochs_range, val_accuracy, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# Save the model as model.h5
model.save('model.h1')
# Load the model
model = load_model('model.h1')

In [ ]:
# Print the class indices 
print("Prediction Dict: ", training_data_frame.class_indices)
# Predict on the test data 
pred = model.predict(test_data_frame)
# Create a class/labels dictionary
classDict = {
            0: "0", 1: "1", 2: "2", 3: "3", 4: "4", 5: "5", 6: "6", 7: "7", 8: "8", 9: "9", 10: "A",
            11: "B", 12: "C", 13: "D", 14: "E", 15: "F", 16: "G", 17: "H", 18: "I", 19: "J", 20: "K",
            21: "L", 22: "M", 23: "N", 24: "O", 25: "P", 26: "Q", 27: "R", 28: "S", 29: "T", 30: "U",
            31: "V", 32: "W", 33: "X", 34: "Y", 35: "Z", 36: "a", 37: "b", 38: "c", 39: "d", 40: "e",
            41: "f", 42: "g", 43: "h", 44: "i", 45: "j", 46: "k", 47: "l", 48: "m", 49: "n", 50: "o",
            51: "p", 52: "q", 53: "r", 54: "s", 55: "t", 56: "u", 57: "v", 58: "w", 59: "x", 60: "y",
            61: "z"}

# Make a data frame that contains the probability for each class
outputDf = pd.DataFrame(pred)
# Get the index of the max probability from the output Data frame
maxIndex = list(outputDf.idxmax(axis=1))
# Print the max index
print("Max index: ", maxIndex)
# Make a loop in range the length of the test data (20)
for i in range(len(test_set)):
    # Read the image 
    image = cv.imread(DATAPATH + '/' + test_set.at[i, 'image'])
    # The title of the plot which is the predicted label
    plt.title(classDict.get(maxIndex[i], "error"))
    # Show the actual image
    plt.imshow(image)
    plt.show()